<a href="https://colab.research.google.com/github/jaalvalcan/GEE_index_sets/blob/main/Example_nir_sent2_era5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-jaalvalcan')

In [ ]:
import geemap
import geopandas as gpd

In [ ]:
# Centralized Settings
AOI_PATH = "/content/sample_data/shitoukoumen.shp"
START_DATE = '2023-10-01'
END_DATE = '2023-10-31'

%pip install pycrs --quiet

ee_fc = geemap.shp_to_ee(AOI_PATH) # Streamlined shapefile loading

Map = geemap.Map()
Map.centerObject(ee_fc, 10)

In [ ]:
# 2. SENTINEL-2 PROCESSING
s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(START_DATE, END_DATE)
    .filterBounds(ee_fc)
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20)))

s2_median = s2_collection.median().clip(ee_fc)

# True Color Visualization
Map.addLayer(s2_median, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'S2 True Color')

In [ ]:
# NIR (Band 8) for Water Contrast
Map.addLayer(s2_median.select('B8'), {'min': 0, 'max': 3000, 'palette': ['blue', 'white']}, 'S2 NIR (B8)')

In [ ]:
# 3. ERA5-LAND TEMPERATURE PROCESSING
era5_land = (ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
    .select('temperature_2m')
    .filterDate(START_DATE, END_DATE)
    .filterBounds(ee_fc))

if era5_land.size().getInfo() > 0:
    # Convert K to C and clip in one step
    temp_c = era5_land.mean().subtract(273.15).clip(ee_fc)

    vis_temp = {
        'min': -5, 'max': 25,
        'palette': ['000080', '0000d9', '4000ff', '8000ff', '0080ff', '00ffff', '00ff80', 'ffff00', 'ff0000']
    }
    Map.addLayer(temp_c, vis_temp, 'Mean Temp Oct 2023 (°C)')
else:
    print("No ERA5 data found for this range.")

In [ ]:
# 4. FINAL DISPLAY
Map.addLayer(ee_fc, {'color': 'red'}, 'AOI Boundary')
Map